In [1]:
from ardac_utils import *
from ipyleaflet import Map, GeoData, GeoJSON, Popup, basemaps
from ipywidgets import Layout, HTML

In [2]:
#use ardac helper functions to climate division and protected area polygons and output geodataframes in 3338
region_gdf = get_area_gdf_by_category("climate_divisions", 3338)
parks_gdf = get_area_gdf_by_category("protected_areas", 3338)

In [3]:
#the three regions of the AK interior are CD3, CD5, and CD6 (Central, Northeast, and Southeast Interior, respectively)
#join the parks that have at least some portion contained within those region polygons, keeping the park geometry
#drop any NA rows to remove parks that are not contained in the interior polygons
interior_region = region_gdf[region_gdf['id']=='CD3'].copy()
interior_parks_gdf = interior_region.sjoin(parks_gdf, how='right', predicate='contains').dropna()
print(interior_parks_gdf)

   index_left id_left         name_left id_right  \
0         0.0     CD3  Central Interior     BLM1   
0         0.0     CD3  Central Interior     BLM5   
0         0.0     CD3  Central Interior     BLM8   
0         0.0     CD3  Central Interior     BLM9   
0         0.0     CD3  Central Interior    BLM12   
0         0.0     CD3  Central Interior    BLM13   
0         0.0     CD3  Central Interior    BLM16   
0         0.0     CD3  Central Interior    FWS12   
0         0.0     CD3  Central Interior    BLM20   
0         0.0     CD3  Central Interior    FWS18   
0         0.0     CD3  Central Interior    BLM24   
0         0.0     CD3  Central Interior    BLM25   
0         0.0     CD3  Central Interior    BLM26   
0         0.0     CD3  Central Interior    BLM27   
0         0.0     CD3  Central Interior    BLM28   
0         0.0     CD3  Central Interior    BLM31   
0         0.0     CD3  Central Interior    BLM33   
0         0.0     CD3  Central Interior    BLM35   
0         0.

In [4]:
#drop & rename columns as required for the get_data_for_gdf_polygons() function
#and get beetles data for the interior parks polygons
#note! some polygons will not return beetles data due to zonal stats function mismatch between polygon size and raster pixel size
interior_parks_gdf.drop(columns=['index_left', 'id_left', 'name_left'], inplace=True)
interior_parks_gdf.rename(columns={'id_right':'id', 'name_right':'name'}, inplace=True)

interior_parks_beetles_data = get_data_for_gdf_polygons(interior_parks_gdf, "beetles")

Bad request, no beetles data for BLM1: Arms Lake Research Natural Area...trying next polygon...
Bad request, no beetles data for BLM8: Galena Mountain Watershed Area Of Critical Environmental Concern...trying next polygon...
Bad request, no beetles data for BLM12: Hogatza Area Of Critical Environmental Concern...trying next polygon...
Bad request, no beetles data for BLM20: Mcquesten Creek Research Natural Area...trying next polygon...
Bad request, no beetles data for BLM24: Nugget Creek Area Of Critical Environmental Concern...trying next polygon...
Bad request, no beetles data for BLM25: Nulato Hills Threatened And Endangered Area...trying next polygon...
Bad request, no beetles data for BLM27: Poss Mountain Area Of Critical Environmental Concern...trying next polygon...
Bad request, no beetles data for BLM28: Redlands Lake Research Natural Area...trying next polygon...
Bad request, no beetles data for BLM31: Snowden Mountain Area Of Critical Environmental Concern...trying next polyg

In [5]:
# center = (interior_parks_gdf.dissolve().centroid.to_crs(4326).y[0], interior_parks_gdf.dissolve().centroid.to_crs(4326).x[0])

# m = Map(
#     basemap=basemaps.Esri.WorldTopoMap,
#     center=center,
#     zoom=6,
#     layout=Layout(width='75%', height='500px'))

# parks_poly = GeoData(geo_dataframe = interior_parks_gdf.to_crs(4326),
#                    style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.01, 'weight':5, 'fillOpacity':0.6},
#                    hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
#                    name = 'Interior Parks and Protected Areas')

# def click_handler(event=None, feature=None, id=None, properties=None):

#     html_popup = HTML()
#     html_popup.value = 'yo!'
#     popup = Popup(
#     location=center,
#     child=html_popup,
#     close_button=True,
#     auto_close=True,
#     close_on_escape_key=False
#     )
#     m.add_layer(popup)


# parks_poly.on_click(click_handler())

# m.add_layer(parks_poly)

# display(m)

In [6]:
subset_df = interior_parks_beetles_data[
    (interior_parks_beetles_data['era']=='2070-2099') & 
    (interior_parks_beetles_data['scenario']=='rcp85')
    ].drop(columns=['era', 'scenario', 'id', 'percent high protection', 'percent minimal protection', 'percent no protection']).reset_index(drop=True)
print(subset_df)

                                                 name       model snowpack  \
0   Dulbi-Kaiyuh Mountains Area Of Critical Enviro...  NCAR-CCSM4      low   
1   Dulbi-Kaiyuh Mountains Area Of Critical Enviro...  NCAR-CCSM4   medium   
2   Dulbi-Kaiyuh Mountains Area Of Critical Enviro...  GFDL-ESM2M      low   
3   Dulbi-Kaiyuh Mountains Area Of Critical Enviro...  GFDL-ESM2M   medium   
4   Dulbi-Kaiyuh Mountains Area Of Critical Enviro...  HadGEM2-ES      low   
5   Dulbi-Kaiyuh Mountains Area Of Critical Enviro...  HadGEM2-ES   medium   
6   Dulbi-Kaiyuh Mountains Area Of Critical Enviro...   MRI-CGCM3      low   
7   Dulbi-Kaiyuh Mountains Area Of Critical Enviro...   MRI-CGCM3   medium   
8   George River Area Of Critical Environmental Co...  NCAR-CCSM4      low   
9   George River Area Of Critical Environmental Co...  NCAR-CCSM4   medium   
10  George River Area Of Critical Environmental Co...  GFDL-ESM2M      low   
11  George River Area Of Critical Environmental Co...  GFDL-ESM2

In [7]:
subset_df_pivot = subset_df.pivot(index= 'name', columns =['model', 'snowpack'], values='climate protection')
subset_df_pivot.columns = subset_df_pivot.columns.to_flat_index().str.join('_')
subset_df_pivot.reset_index(inplace=True)
print(subset_df_pivot)

                                                name NCAR-CCSM4_low  \
0  Dulbi-Kaiyuh Mountains Area Of Critical Enviro...           high   
1  George River Area Of Critical Environmental Co...        minimal   
2  Indian River Watershed Area Of Critical Enviro...           high   
3   Jim River Area Of Critical Environmental Concern           high   
4                    Kanuti National Wildlife Refuge           high   
5                   Nowitna National Wildlife Refuge           high   
6  Oskawalik River Area Of Critical Environmental...        minimal   

  NCAR-CCSM4_medium GFDL-ESM2M_low GFDL-ESM2M_medium HadGEM2-ES_low  \
0              none           high           minimal           high   
1              none           high              high           high   
2           minimal           high              high           high   
3           minimal           high              high           high   
4           minimal           high           minimal           high   
5    

In [8]:
map_gdf = interior_parks_gdf.merge(
    subset_df_pivot,
    how='left', on='name').dropna()
center = (map_gdf.dissolve().centroid.to_crs(4326).y[0], map_gdf.dissolve().centroid.to_crs(4326).x[0])
parks_poly = GeoData(geo_dataframe = map_gdf.to_crs(4326))

map = Map(
    basemap=basemaps.Esri.WorldTopoMap,
    center=center,
    zoom=6,
    layout=Layout(width='90%', height='500px'))

for feature in parks_poly.data['features']:
    geojson = GeoJSON(data=feature, 
            style={'color': 'black', 'fillColor': 'blue', 'opacity':0.1, 'weight':5, 'fillOpacity':0.6},
            hover_style={'fillColor': 'lightblue' , 'fillOpacity': 0.2},
            )
    name = feature['properties']['name']
   
    html = """
      <h4><b>""" + "".join(name) + ": 2070-2099, RCP 8.5" + """</b></h4>
      <br>
      <font size="0.7">
      <table>
        <tr>
          <th>NCAR-CCSM4: low snowpack</th>
          <th>NCAR-CCSM4: medium snowpack</th>
          <th>GFDL-ESM2M: low snowpack</th>
          <th>GFDL-ESM2M: medium snowpack</th>
          <th>HadGEM2-ES: low snowpack</th>
          <th>HadGEM2-ES: medium snowpack</th>
          <th>MRI-CGCM3: low snowpack</th>
          <th>MRI-CGCM3: medium snowpack</th>
        </tr>
        <tr>
          <td>""" + "".join(feature['properties']['NCAR-CCSM4_low']) + """</td>
          <td>""" + "".join(feature['properties']['NCAR-CCSM4_medium']) + """</td>
          <td>""" + "".join(feature['properties']['GFDL-ESM2M_low']) + """</td>
          <td>""" + "".join(feature['properties']['GFDL-ESM2M_medium']) + """</td>
          <td>""" + "".join(feature['properties']['HadGEM2-ES_low']) + """</td>
          <td>""" + "".join(feature['properties']['HadGEM2-ES_medium']) + """</td>
          <td>""" + "".join(feature['properties']['MRI-CGCM3_low']) + """</td>
          <td>""" + "".join(feature['properties']['MRI-CGCM3_medium']) + """</td>
        </tr>
      </table>
      </font>
      """

    message1 = HTML(layout = Layout(width='600px'))
    message1.value = html

    # Popup with a given location on the map:
    popup = Popup(
        location=center,
        child=message1,
        close_button=True,
        auto_close=True,
        minWidth=700
        )

    geojson.popup = popup
    map.add_layer(geojson)

display(map)

Map(center=[64.83757887332173, -153.88476806788645], controls=(ZoomControl(options=['position', 'zoom_in_text'…